In [ ]:
from IPython.core.display import display, HTML
%config IPCompleter.greedy = True
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [ ]:
# Import libraries and define constants

import os
from datetime import datetime
from free_bwacs_model import FreeBWACS
from src.helpers import sorted_alphanumeric, save_results_on_file

RUNS = 10
SAVE_RESULTS = True
SAVE_IMGS = False
RESULTS_FILE_WITH_HEADER = True
BASE_RESULTS_FOLDER = 'results'

In [ ]:
# Execution of runs for instances files for FREE-ANT

instances_with_errors = []
instances_folder = 'instances/CVRPLIB/CMT'
instances = sorted_alphanumeric([instance_name.replace(
    '.vrp', '') for instance_name in os.listdir(instances_folder)])
# instances = ['CMT12']
cluster_type = 'kmeans'  # 'kmeans' or 'kmedoids'
ant_type = 'fa'  # 'fa', 'ra' or 'random'
now = datetime.now()
this_execution_folder = now.strftime("%Y%m%d_%H%M%S")

# Validation of folder to save results
if SAVE_RESULTS:
    if not os.path.exists(BASE_RESULTS_FOLDER):
        os.makedirs(BASE_RESULTS_FOLDER)

# Start running the algorithm
for instance in instances:
    
    solution_list = []
    results_folder = f'{this_execution_folder}/{instance}'
    file_name = f'{str(instance).lower()}_{ant_type}_{cluster_type}'

    if SAVE_IMGS: os.makedirs(f'{BASE_RESULTS_FOLDER}/{results_folder}/imgs', exist_ok=True)
    
    for run in range(RUNS):
        try:
            bwacs = FreeBWACS(
                instance=f'{instances_folder}/{instance}',
                max_nodes=9999,
                cluster_type=cluster_type,
                metric='euclidian',
                tare_percentage=0.15,
                max_iterations=100,
                total_ant_divider=1,
                only_k_optimum=True,
                start_ant_on_best_nodes=True,
                # heuristic_type will let you choice what information gotta build the heuristic information matrix
                # 0 - Only distances data / Best beta: 5
                # 1 - Only energies data / Best gamma: 5
                # 2 - Distances data * energies data / Best parameters: 5 - 5
                # 3 - Only distances data of Savings algorithm / Best delta: 1
                # 4 - Only energies data of Savings algorithm / Best eta: 2
                # 5 - Distances data * distances savings data / Best parameters: 4 - 5
                # 6 - Energies data * energies savings data / Best parameters: 5 - 5
                # 7 - Distances data * energies data * distances savings data / Best parameters: 3 - 4 - 3
                # 8 - Distances data * energies data * energies savings data / Best parameters: 3 - 4 - 3
                # 9 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                # 10 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                # 11 - Distances data * energies data * distances savings data * energies savings data * capacity utilization data / Best parameters: 2 - 4 - 3 - 4 - 1
                heuristic_type=2,
                # Importance of pheromones trace. Recommended alpha value in: [1, 4]
                alpha=3,
                # Importance of distance heuristic information. Recommended beta value in: [2, 5]
                beta=2,
                # Importance of energy heuristic information. Recommended gamma value in: [2, 5]
                gamma=4,
                # Importance of distances's Savings Algorithm information. Recommended delta value in: [0, 2]
                delta=1,
                # Importance of energies's Savings Algorithm information. Recommended eta value in: [0, 2]
                eta=1,
                # Importance of vehicle's capacity utilization. Recommended mi value in: [0, 2]
                mi=0,
                # Define the strategy used for pheromone updating. - options: | 0 - 1 |
                # The value 0 is for the classic function in ACO: 1 / Best_Solution_Quality
                # The value 1 if for the improve function know as Ant Weight Strategy.
                pheromone_updating_strategy=0,
                local_ant_update_pheromones=False,
                best_iteration_ant_update_pheromones=False,
                best_global_ant_update_pheromones=True,
                penalize_worst_solution=True,
                mutate_pheromones_matrix=True,
                # Evaporation rate of the pheromones by the next function: phoromone =  (1 - p) * pheromone. Recommended value for p: 0.02.
                p=0.02,
                # Mutation probability for every row of the pheromones matrix, such as: mutate if (random number between 0 and 1) <= Pm
                # Recommended value for Pm: 0.3.
                Pm=0.3,
                # Mutation intensity for the pheromones matrix. Recommended value for sigma in: [2, 4]
                sigma=3,
                # Constant used in Pseudorandom Transition Rule. This is a probability of choice the next node by argmax, define in Ant Colony System.
                # Recommended value for l0 in: [0.2, 0.4]
                l0=0.25,
                # Constant used for a lot of functions, such as: f(.) = H / specific_values.
                H=1,
                ls_ant_solution=False,
                ls_best_iteration=True,
                ls_best_global=False,
                use_normalized_matrix=False,
                print_instance=False,
                print_clusters=False,
                print_solution=False,
                print_distance_matrix=False,
                print_energy_matrix=False,
                print_distance_saving_matrix=False,
                print_energy_saving_matrix=False,
                print_combination_matrix=False,
                print_pheromone_matrix=False,
                output_sol_img=f'{BASE_RESULTS_FOLDER}/{results_folder}/imgs/{file_name}.png'
            )

            solution_energy, solution_arcs, solution_distance, solution_time = bwacs.solve()
            solution_list.append(
                (solution_energy, solution_arcs, solution_distance, solution_time))
        except Exception as e:
            instances_with_errors.append((instance, e))
            break
    
    if (len(solution_list) > 0): 
        print(f'Solutions for instance {instance}: {solution_list}')
        print(f'{RUNS} runs for instance {instance} finished at {datetime.now().strftime("%Y%m%d_%H%M%S")}')
        print(
            f'Best solution for instance {instance}: {min(solution_list, key=lambda x: x[0])}')
        print(
            f'Worst solution for instance {instance}: {max(solution_list or [], key=lambda x: x[0])}')
        

        if SAVE_RESULTS:
            os.makedirs(
                f'{BASE_RESULTS_FOLDER}/{results_folder}', exist_ok=True)
            save_results_on_file(f'{BASE_RESULTS_FOLDER}/{results_folder}', file_name, ant_type, cluster_type,
                                solution_list, instance, 'csv', 'energy,arcs,distance,time(ms)')

if  len(instances_with_errors) > 0: 
    print(f'Instances with errors: {instances_with_errors}')
    error_file = open(f'{BASE_RESULTS_FOLDER}/{this_execution_folder}/error_instances.txt', 'a')
    for instance, error in instances_with_errors:
        error_file.write(f'{instance}: {error}\n')
    error_file.close()
